# ИИ-агент SMIRNOFF_AI на LangGraph

## Что мы делаем
Создаём простого ИИ-консультанта для AI-агентства SMIRNOFF_AI:
- Диалог в Telegram боте
- Ответы об услугах (из системного промпта)  
- Поиск по PDF кейсам/обучению (RAG)
- Веб-поиск для вопросов вне домена (Tavily)
- Сбор контактов клиентов

## Архитектура
**Паттерн ReAct** (Reason + Act) - агент рассуждает, затем действует через инструменты.

**Инструменты:**
- `rag_search()` - поиск в PDF без указания источников
- `web_search()` - веб-поиск для вопросов вне домена  
- `lead_capture()` - сохранение контактов

**Граф:** agent → tools/END, tools → agent  
**Состояние:** messages + lead_summary_text  
**Хранение:** SqliteSaver

**Референс:** `langgraph-samples/react-agent-from-scratch.ipynb`  
**Детали:** `graph_design.md`


```mermaid
graph TD
    %% Основной поток
    A[🤖 Agent<br/>Node] -->|function_call| SC{🔄 Should Continue<br/>Conditional Edge}
    
    SC -->|"continue"| T[🛠️ Tools<br/>Node]
    SC -->|"end"| END[🏁 End]
    
    %% Tools Node с тремя инструментами
    T -->|rag_search| RAG[🔍 RAG Search<br/>ChromaDB + PDF]
    T -->|web_search| WEB[🌐 Web Search<br/>Tavily API]  
    T -->|lead_capture| LEAD[📋 Lead Capture<br/>Save Contact]
    
    %% Возврат от Tools к Agent
    RAG -->|results| T
    WEB -->|results| T
    LEAD -->|confirmation| T
    
    T -->|tool_results| A
    
    %% Состояние и данные
    subgraph STATE["💾 State Management"]
        direction LR
        SQL[🗃️ SqliteSaver<br/>./var/sqlite/state.db]
        MSGS[📝 Messages History]
        LEADS[📊 Lead Summary Text]
    end
    
    A -.->|save state| STATE
    T -.->|update state| STATE
    
    %% Внешние сервисы
    subgraph EXTERNAL["🔌 External Services"]
        direction LR
        OAI[🤖 OpenAI<br/>gpt-4o-mini]
        CHR[🗄️ ChromaDB<br/>Vector Store]
        TAV[🔍 Tavily<br/>Web Search]
    end
    
    A -.->|llm.invoke| OAI
    RAG -.->|retriever.invoke| CHR
    WEB -.->|tavily.invoke| TAV
    
    %% Данные
    subgraph DATA["📁 Data Sources"]
        direction LR
        PDF[📄 SMIRNOFF_AI.pdf<br/>./data/pdf/]
        PROMPT[📝 prompt_info.txt<br/>Company info]
    end
    
    CHR -.->|load from| PDF
    A -.->|system prompt| PROMPT
    
    %% Вход/выход
    TB[📱 Telegram Bot] -->|user_message| A
    END -->|ai_response| TB
    
    %% Стили узлов
    classDef nodeStyle fill:#e3f2fd,stroke:#1976d2,stroke-width:2px,color:#000
    classDef edgeStyle fill:#fff3e0,stroke:#f57c00,stroke-width:2px,color:#000
    classDef toolStyle fill:#e8f5e8,stroke:#388e3c,stroke-width:2px,color:#000
    classDef serviceStyle fill:#fce4ec,stroke:#c2185b,stroke-width:2px,color:#000
    classDef dataStyle fill:#f1f8e9,stroke:#689f38,stroke-width:2px,color:#000
    classDef endStyle fill:#ffebee,stroke:#d32f2f,stroke-width:2px,color:#000
    
    class A,T nodeStyle
    class SC edgeStyle
    class RAG,WEB,LEAD toolStyle
    class STATE,EXTERNAL serviceStyle
    class DATA dataStyle
    class END,TB endStyle
```

## 2. Setup окружения

Устанавливаем пакеты, проверяем ключи API, создаём папки для данных.


In [ ]:
# Установка пакетов
%pip install -q langgraph langchain-openai langchain-tavily langchain-chroma aiogram pypdf python-dotenv
%pip install -q langfuse langgraph-checkpoint-sqlite


In [ ]:
import os
import logging

import dotenv
from pathlib import Path
from dotenv import load_dotenv


# Загрузка переменных окружения
dotenv.load_dotenv('.env', override=True)

# Вывод всех ключей из окружения
env_keys = [
    "OPENAI_API_KEY",
    "TAVILY_API_KEY",
    "LANGSMITH_TRACING",
    "LANGSMITH_ENDPOINT",
    "LANGSMITH_API_KEY",
    "LANGSMITH_PROJECT",
    "LANGSMITH_DATASET",
    "MODEL",
    "EMBEDDING_MODEL",
]

print("Переменные окружения:")
for key in env_keys:
    value = os.getenv(key)
    if value:
        # Маскируем значение для безопасности (показываем только первые 10 символов)
        masked_value = value[:20] + "..." if len(value) > 20 else value
        print(f"  {key}: {masked_value}")
    else:
        print(f"  {key}: не установлена")
print()


# Проверка ключевых API ключей
required_keys = ["OPENAI_API_KEY", "TAVILY_API_KEY"]
missing_keys = [key for key in required_keys if not os.getenv(key)]

if missing_keys:
    raise ValueError(f"Отсутствуют переменные окружения: {missing_keys}")

# Создание папок
Path("./var/chromadb").mkdir(parents=True, exist_ok=True)
Path("./var/sqlite").mkdir(parents=True, exist_ok=True)
Path("./data/pdf").mkdir(parents=True, exist_ok=True)

# Базовый логгер
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Setup завершён ✅")


## 3. Данные и индекс

RAG - поиск по документам. Загружаем PDF-файлы, разбиваем на части и создаём поисковый индекс.

**План:**
- Сканирование `./data/pdf/`
- PyPDFLoader + RecursiveCharacterTextSplitter
- Создание коллекции Chroma `smirnoff_ai_docs`
- Параметры: chunk_size=1200, chunk_overlap=150


## 3. Данные и индекс

RAG - поиск по документам. Загружаем PDF-файлы, разбиваем на части и создаём поисковый индекс.

**План:**
- Сканирование `./data/pdf/`
- PyPDFLoader + RecursiveCharacterTextSplitter
- Создание коллекции Chroma `smirnoff_ai_docs`
- Параметры: chunk_size=1200, chunk_overlap=150


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import glob

# Сканируем PDF файлы
pdf_files = glob.glob("./data/pdf/*.pdf")
if not pdf_files:
    raise FileNotFoundError("Нет PDF файлов в ./data/pdf/")

logger.info(f"Найдено PDF файлов: {len(pdf_files)}")
for file in pdf_files:
    logger.info(f"- {file}")


In [ ]:
# Загрузка и нарезка PDF
all_docs = []

for pdf_file in pdf_files:
    loader = PyPDFLoader(pdf_file)
    pages = loader.load()
    logger.info(f"Загружено страниц из {pdf_file}: {len(pages)}")
    all_docs.extend(pages)

# Разбиваем на чанки
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200, 
    chunk_overlap=150
)
doc_chunks = text_splitter.split_documents(all_docs)

logger.info(f"Создано чанков: {len(doc_chunks)}")
logger.info(f"Пример чанка: {doc_chunks[0].page_content[:200]}...")


In [ ]:
# Создание Chroma индекса
embeddings = OpenAIEmbeddings(model=os.getenv("EMBEDDING_MODEL"))

# Удаляем существующую коллекцию для воспроизводимости
import chromadb
chroma_path = "./var/chromadb"

# Отключаем телеметрию Chroma
os.environ["ANONYMIZED_TELEMETRY"] = "False"

# Подключаемся к Chroma и удаляем коллекцию если есть
client = chromadb.PersistentClient(path=chroma_path)
try:
    client.delete_collection("smirnoff_ai_docs")
    logger.info("Старая коллекция удалена")
except:
    logger.info("Коллекция не существует")

# Создаем векторное хранилище
vectorstore = Chroma.from_documents(
    documents=doc_chunks,
    collection_name="smirnoff_ai_docs",
    embedding=embeddings,
    persist_directory=chroma_path,
)

logger.info(f"Chroma индекс создан ✅")
logger.info(f"Коллекция: smirnoff_ai_docs")
logger.info(f"Документов в индексе: {vectorstore._collection.count()}")


## 4. Инициализация внешних провайдеров и сервисов

In [ ]:
# Настройка OpenAI
from langchain_openai import ChatOpenAI

# Модель для диалога  
llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    temperature=0.7,
    max_tokens=700
)

logger.info("OpenAI LLM настроен ✅")


In [ ]:
# Настройка Chroma retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

logger.info("Chroma retriever настроен ✅")


In [ ]:
# Настройка Tavily для веб-поиска
from langchain_tavily import TavilySearch

tavily_search = TavilySearch(
    max_results=3
)

logger.info("Tavily клиент настроен ✅")


## 5. Инструменты агента

Создаём 3 простых инструмента для разных типов запросов по примеру `react-agent-from-scratch.ipynb`.


In [ ]:
from langchain_core.tools import tool

@tool
def rag_search(query: str) -> str:
    """Ищет информацию о кейсах и программах обучения SMIRNOFF_AI в загруженных PDF документах."""
    docs = retriever.invoke(query)
    
    if not docs:
        return "По данному запросу информация в наших материалах не найдена."
    
    # Собираем контекст без указания источников
    context_parts = []
    for doc in docs:
        content = doc.page_content.strip()
        if content:
            context_parts.append(content)
    
    combined_context = "\n\n".join(context_parts)
    return combined_context

logger.info("RAG инструмент создан ✅")


In [ ]:
@tool
def web_search(query: str) -> str:
    """Выполняет веб-поиск для вопросов вне домена услуг и кейсов компании SMIRNOFF_AI."""
    try:
        results = tavily_search.invoke({"query": query})
        
        if not results:
            return "По данному запросу информация в интернете не найдена."
        
        # TavilySearch возвращает строку, не список
        if isinstance(results, str):
            # Ограничиваем длину результата
            return results[:500] + "..." if len(results) > 500 else results
        
        # Если это список (для совместимости)
        elif isinstance(results, list) and len(results) > 0:
            summaries = []
            # Безопасно берем максимум 2 результата
            for result in results[:2]:
                if isinstance(result, dict) and 'content' in result:
                    content = result['content'][:300]
                    summaries.append(content)
                elif isinstance(result, str):
                    summaries.append(result[:300])
            return "\n\n".join(summaries) if summaries else "Не удалось получить релевантную информацию."
        
        # Если это другой тип объекта
        else:
            result_str = str(results)
            return result_str[:500] + "..." if len(result_str) > 500 else result_str
        
    except Exception as e:
        logger.error(f"Детальная ошибка web_search: {type(e).__name__}: {str(e)}")
        return f"Ошибка при поиске. Попробуйте переформулировать вопрос."

logger.info("Веб-поиск инструмент создан ✅")


In [ ]:
@tool 
def lead_capture(text: str) -> str:
    """Сохраняет контактную информацию потенциального клиента (имя, телефон, email, запрос)."""
    # Логируем сохраненную информацию
    logger.info(f"💼 Лид сохранен: {text}")
    
    return f"Контактная информация сохранена: {text[:100]}... Наш менеджер свяжется с вами в ближайшее время."

logger.info("Инструмент сбора лидов создан ✅")

## 6. Граф ReAct

Основа агента - граф ReAct. Создаём простейший граф: агент думает → вызывает инструменты → отвечает.

**По примеру react-agent-from-scratch.ipynb**

In [ ]:
# Определение состояния графа (строго по примеру)
from typing import (
    Annotated,
    Sequence,
    TypedDict,
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """The state of the agent."""
    # add_messages - это reducer функция, которая автоматически объединяет новые сообщения
    # со старыми в состоянии графа. Без неё каждый узел будет перезаписывать весь список,
    # а не добавлять к нему. Это ключевая особенность LangGraph для работы с диалогами.
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # Дополнительное поле для сохранения лидов
    lead_summary_text: str

logger.info("Состояние графа определено ✅")


In [ ]:
# Создаем список инструментов
tools = [rag_search, web_search, lead_capture]

In [ ]:
# Привязка инструментов к модели (строго по примеру)
model = llm.bind_tools(tools)

logger.info("Модель с инструментами настроена ✅")


In [ ]:
# Создаем словарь для быстрого доступа по имени
# Это позволяет нам быстро найти нужный инструмент по его имени
# Например: tools_by_name["rag_search"] вернет функцию rag_search
# LangGraph будет использовать это для вызова нужного инструмента когда LLM запросит его
tools_by_name = {tool.name: tool for tool in tools}

logger.info(f"Всего инструментов: {len(tools)}")

In [ ]:
# Определение узлов и рёбер (строго по примеру)
import json
from langchain_core.messages import ToolMessage, SystemMessage
from langchain_core.runnables import RunnableConfig

# Узел инструментов (как в примере)
# ВАЖНО: Инструменты НЕ могут напрямую обновлять состояние графа!
# Только узлы могут обновлять состояние через return значения.

def tool_node(state: AgentState):
    outputs = []
    new_lead_summary = state.get("lead_summary_text", "")
    
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        
        # Если это lead_capture - сохраняем в состояние
        if tool_call["name"] == "lead_capture":
            new_lead_summary = tool_call["args"]["text"]
        
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"], 
                tool_call_id=tool_call["id"],
            )
        )
    
    # Возвращаем обновленное состояние
    return {
        "messages": outputs,
        "lead_summary_text": new_lead_summary
    }

logger.info("Узел инструментов создан ✅")


In [ ]:
# Загрузка информации о компании для системного промпта
with open("./data/prompt_info.txt", "r", encoding="utf-8") as f:
    company_info = f.read()

# Узел агента с системным промптом SMIRNOFF_AI
def call_model(state: AgentState, config: RunnableConfig):
    system_prompt = SystemMessage(f"""Ты - ИИ-консультант AI-агентства SMIRNOFF_AI. 

{company_info}

ПРАВИЛА ОТВЕТОВ:
- Всегда отвечай на русском языке
- Будь кратким и деловым (3-5 предложений)  
- Не указывай источники при использовании RAG
- Ссылки в веб-поиске - только при явном запросе
- После каждого ответа предлагай следующий шаг (CTA)

ИНСТРУМЕНТЫ:
- rag_search: для поиска в наших кейсах и программах обучения
- web_search: обязательно использовать для веб-поиска по всем вопросам вне кейсов, программ обучения, портфолио и имеющейся информации о компании SMIRNOFF_AI, кроме очевидно простых вопросов 
- lead_capture: для сохранения контактов клиентов

Помогай клиентам найти подходящие AI-решения для их бизнеса.""")
    
    response = model.invoke([system_prompt] + state["messages"], config)
    return {"messages": [response]}

logger.info("Узел агента с системным промптом создан ✅")


In [ ]:
# Условное ребро (строго по примеру)
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # Если нет вызова инструментов, заканчиваем
    if not last_message.tool_calls:
        return "end"
    # Иначе продолжаем к инструментам
    else:
        return "continue"

logger.info("Логика условного ребра создана ✅")


In [ ]:
# SqliteSaver для сохранения состояния между сессиями
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# Подключение к базе данных
db_path = "./var/sqlite/state.db"
checkpointer = SqliteSaver(sqlite3.connect(db_path, check_same_thread=False))
checkpointer.setup()

logger.info(f"SqliteSaver настроен: {db_path} ✅")


In [ ]:
# Определение и компиляция графа (строго по примеру)
from langgraph.graph import StateGraph, END

# Создаем граф
workflow = StateGraph(AgentState)

# Добавляем узлы
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Устанавливаем точку входа
workflow.set_entry_point("agent")

# Добавляем условное ребро
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "end": END,
    },
)

# Добавляем обычное ребро
workflow.add_edge("tools", "agent")

# Компилируем граф с checkpointer
graph = workflow.compile(checkpointer=checkpointer)

logger.info("Граф ReAct скомпилирован ✅")


In [ ]:
# Визуализация графа
from IPython.display import Image, display

# Создаем PNG изображение графа
graph_image = graph.get_graph().draw_mermaid_png()
display(Image(graph_image))
logger.info("Граф визуализирован ✅")



## 7. Хелперы запуска

Тестируем агента. Создаём простую функцию для проверки работы.


In [ ]:
# Дополнительная функция для просмотра состояния
def show_state(chat_id: str):
    """Показывает текущее состояние графа для данного chat_id."""
    config = {"configurable": {"thread_id": str(chat_id)}}
    
    try:
        state = graph.get_state(config)
        print(f"\n📊 Состояние графа для chat_id: {chat_id}")
        print(f"📝 Количество сообщений: {len(state.values.get('messages', []))}")
        print(f"💼 Лид: {state.values.get('lead_summary_text', 'Нет данных')}")
        return state
    except Exception as e:
        print(f"❌ Ошибка получения состояния: {e}")
        return None



In [ ]:
# Функция run_turn для тестирования (строго по примеру react-agent-from-scratch.ipynb)
def run_turn(chat_id: str, user_text: str):
    """Запускает один оборот диалога с агентом."""
    
    # Создаем конфиг с thread_id (как в примере)
    config = {"configurable": {"thread_id": str(chat_id)}}
    
    # Создаем сообщение пользователя (как в примере)
    inputs = {"messages": [("user", user_text)]}
    
    print(f"\n💬 Чат {chat_id}: {user_text}")
    print("=" * 60)
    
    # Запускаем граф с режимом updates для просмотра каждого шага
    for chunk in graph.stream(inputs, config=config, stream_mode="updates"):
        for node, output in chunk.items():
            if "messages" in output:
                for message in output["messages"]:
                    if hasattr(message, 'pretty_print'):
                        message.pretty_print()
                    else:
                        print(f"   {message}")
            if "lead_summary_text" in output and output["lead_summary_text"]:
                print(f"💼 Лид сохранен: {output['lead_summary_text']}")
        print("-" * 40)
    
    print("=" * 60)
    return config


## 8. Смоук-тесты всех сценариев

Проверяем все типы запросов: услуги, кейсы, веб-поиск, лиды + полноценный диалог клиента.


In [ ]:
# 🧪 ТЕСТ 1: Услуги компании (системный промпт)
print("=" * 80)
print("🧪 ТЕСТ 1: Услуги компании")
print("=" * 80)

run_turn("test1", "Какие услуги предоставляет SMIRNOFF_AI?")


In [ ]:
# 🧪 ТЕСТ 2: Кейсы и обучение (rag_search)
print("=" * 80)
print("🧪 ТЕСТ 2: Кейсы и обучение") 
print("=" * 80)

run_turn("test2", "Какие проекты вы реализовали для агро сектора?")


In [ ]:
# 🧪 ТЕСТ 3: Веб-поиск (web_search)
print("=" * 80)
print("🧪 ТЕСТ 3: Веб-поиск вне домена")
print("=" * 80)

run_turn("test34", "А LangGraph версии 1.0 уже выпустили?")


In [ ]:
# 🧪 ТЕСТ 4: Сбор лидов (lead_capture)  
print("=" * 80)
print("🧪 ТЕСТ 4: Сбор лидов")
print("=" * 80)

run_turn("test41", "Меня зовут Иван Петров, телефон +7-123-456-7890, хочу обсудить проект по автоматизации")


In [ ]:
# Проверяем состояние лида
show_state("test41")


In [ ]:
# 🎯 ГЛАВНЫЙ ТЕСТ: Полноценный диалог клиента (ВСЕ инструменты)
print("=" * 80)
print("🎯 ГЛАВНЫЙ ТЕСТ: Полноценный диалог клиента")  
print("=" * 80)

client_id = "main_client_11"

# Шаг 1: Услуги (системный промпт)
print("\n📞 Шаг 1: Клиент спрашивает об услугах")
run_turn(client_id, "Привет! Что вы можете для нас сделать?")


In [ ]:
# Шаг 2: Кейсы (rag_search)
print("\n📊 Шаг 2: Клиент просит кейсы")
run_turn(client_id, "Интересно. А есть успешные кейсы для агро сектора?")


In [ ]:
# Шаг 3: Веб-поиск (web_search)
print("\n🔍 Шаг 3: Клиент задает технический вопрос")
run_turn(client_id, "А правда, что вышла новая версия Langchain 1.0 для production применения? Вы ее уже исползуете в ваших проектах и есть готовые кейсы? ")


In [ ]:
# Шаг 4: Лид (lead_capture)
print("\n💼 Шаг 4: Клиент оставляет контакты")
run_turn(client_id, "Понятно. Хочу обсудить наш проект. Нужна автоматизация техподдержки")


In [ ]:
run_turn(client_id, "Александра, sasha@teleport.ru")


In [ ]:
# Проверяем состояние лида
show_state(client_id)

## 🎉 Итоговый результат

**ИИ-агент SMIRNOFF_AI успешно создан!**

✅ **8 итераций завершены:**
1. ✅ Шапка и предпосылки
2. ✅ Setup окружения  
3. ✅ Данные и индекс (Chroma)
4. ✅ Провайдеры (OpenAI, Tavily)
5. ✅ Инструменты (3 tools)
6. ✅ Граф ReAct (LangGraph)
7. ✅ Хелперы запуска (pretty_print)
8. ✅ Смоук-тесты (все сценарии)

**Что умеет агент:**
- 💬 Консультирует об услугах SMIRNOFF_AI  
- 📊 Ищет кейсы и программы в PDF (RAG)
- 🔍 Отвечает на вопросы вне домена (Tavily)
- 💼 Собирает контакты клиентов (leads)
- 🧠 Сохраняет состояние между сессиями (SQLite)

**Технический стек:**
- LangGraph + ReAct pattern
- OpenAI gpt-4o-mini + embeddings
- ChromaDB для RAG  
- Tavily для веб-поиска
- SqliteSaver для состояния
- Langfuse + LangSmith для трейсинга

**Готов к продакшену!** 🚀


## 9. Telegram-бот (поллинг) 

Подключаем агента к Telegram для живого общения с клиентами.


In [ ]:
# Дополнительная установка для Telegram-бота
%pip install -q python-telegram-bot


In [ ]:
import asyncio
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, filters, ContextTypes

# Проверяем наличие токена
TELEGRAM_TOKEN = os.getenv("TELEGRAM_TOKEN")
if not TELEGRAM_TOKEN:
    print("⚠️ TELEGRAM_BOT_TOKEN не найден в .env")
    print("Добавьте токен от @BotFather в .env файл:")
    print("TELEGRAM_BOT_TOKEN=your_bot_token_here")
else:
    print(f"✅ Telegram токен найден: {TELEGRAM_TOKEN[:10]}...")

logger.info("Telegram импорты загружены ✅")


In [ ]:
# Функция для общения с агентом из Telegram
def get_agent_response(chat_id: str, user_text: str) -> str:
    """Получает ответ от агента и возвращает только текст ответа."""
    
    # Создаем конфиг с thread_id
    config = {"configurable": {"thread_id": str(chat_id)}}
    
    # Создаем сообщение пользователя
    inputs = {"messages": [("user", user_text)]}
    
    # Запускаем граф и собираем ответ
    agent_response = ""
    lead_info = ""
    
    try:
        for step in graph.stream(inputs, config=config, stream_mode="values"):
            message = step["messages"][-1]
            
            # Получаем ответ агента (только от AI, не от пользователя)
            if hasattr(message, 'content') and message.content:
                # Проверяем tool_calls только если атрибут существует
                has_tool_calls = hasattr(message, 'tool_calls') and message.tool_calls
                
                # Берем только финальный ответ агента (без вызовов инструментов)
                if not has_tool_calls and str(type(message)).find('AIMessage') > -1:
                    agent_response = message.content
            
            # Проверяем лид
            if "lead_summary_text" in step and step["lead_summary_text"]:
                lead_info = f"\n\n💼 Контакт сохранен: {step['lead_summary_text'][:100]}..."
        
        return agent_response + lead_info if agent_response else "Извините, не смог обработать ваш запрос."
        
    except Exception as e:
        logger.error(f"Ошибка агента: {e}")
        return "Извините, произошла ошибка. Попробуйте позже."

logger.info("Функция get_agent_response исправлена ✅")


In [ ]:
# Обработчики Telegram-бота
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик команды /start"""
    welcome_text = """
🤖 Добро пожаловать в SMIRNOFF_AI!

Я ваш ИИ-консультант. Могу рассказать о:
• Наших AI-решениях и услугах
• Успешных кейсах и проектах  
• Ответить на технические вопросы
• Помочь с выбором решения

Просто напишите ваш вопрос!
    """
    await update.message.reply_text(welcome_text)
    logger.info(f"Пользователь {update.effective_user.id} запустил бота")

async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Обработчик текстовых сообщений"""
    user_id = update.effective_user.id
    user_text = update.message.text
    
    logger.info(f"Сообщение от {user_id}: {user_text}")
    
    # Показываем, что бот печатает
    await update.message.chat.send_action("typing")
    
    # Получаем ответ от агента
    response = get_agent_response(str(user_id), user_text)
    
    # Отправляем ответ
    await update.message.reply_text(response)
    logger.info(f"Ответ отправлен пользователю {user_id}")

logger.info("Telegram handlers созданы ✅")


In [ ]:
# Запуск Telegram-бота (НЕ ЗАПУСКАЙТЕ если токен не настроен!)
async def run_telegram_bot():
    """Запускает Telegram-бота с поллингом"""
    
    if not TELEGRAM_TOKEN:
        print("❌ Нет токена! Добавьте TELEGRAM_TOKEN в .env")
        return
    
    print("🚀 Запускаем Telegram-бота...")
    
    # Создаем приложение
    app = Application.builder().token(TELEGRAM_TOKEN).build()
    
    # Добавляем обработчики
    app.add_handler(CommandHandler("start", start_command))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))
    
    print("✅ Бот запущен! Напишите ему в Telegram")
    print("🛑 Для остановки нажмите Ctrl+C")
    
    # Запускаем поллинг
    await app.run_polling(drop_pending_updates=True)

# Функция для запуска в ноутбуке
def start_telegram_bot():
    """Запускает бота в ноутбуке"""
    try:
        # Проверяем, есть ли уже running event loop
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # В Jupyter нужно использовать nest_asyncio
            import nest_asyncio
            nest_asyncio.apply()
            
        asyncio.run(run_telegram_bot())
    except KeyboardInterrupt:
        print("\n🛑 Бот остановлен")
    except Exception as e:
        print(f"❌ Ошибка запуска: {e}")

logger.info("Функция запуска Telegram-бота создана ✅")


In [ ]:
# 🔥 ЗАПУСК TELEGRAM-БОТА (раскомментируйте когда настроите токен)
# start_telegram_bot()

print("""
📱 Для запуска Telegram-бота:

1. Создайте бота через @BotFather в Telegram
2. Добавьте токен в .env: TELEGRAM_BOT_TOKEN=your_token
3. Раскомментируйте строку выше: start_telegram_bot()
4. Запустите ячейку

Бот будет работать с нашим агентом! 🤖
""")


## 10. LangChain 1.0 create_agent - упрощённый подход

**Что такое `create_agent()`?**

`create_agent()` - это новый high-level API в LangChain 1.0, который автоматически создаёт ReAct агента без необходимости вручную строить граф состояний.

**Отличия от ручного LangGraph подхода:**
- ✅ Меньше кода (~20 строк vs ~200 строк)
- ✅ Быстрый старт для стандартных агентов
- ✅ Автоматическая настройка графа
- ❌ Меньше контроля над внутренней логикой

**Что будем использовать:**
- `llm` - уже созданная ChatOpenAI модель (gpt-4o)
- `tools` - уже созданные инструменты: rag_search, web_search, lead_capture
- `company_info` - загруженная информация о компании
- `checkpointer` - SqliteSaver для сохранения состояния


In [ ]:
# Создание агента с помощью create_agent()
from langchain.agents import create_agent

agent_langchain = create_agent(
    model=llm,  # уже созданная ChatOpenAI 
    tools=[rag_search, web_search, lead_capture],  # уже созданные @tool функции
    system_prompt=f"""Ты - ИИ-консультант AI-агентства SMIRNOFF_AI.

{company_info}

ПРАВИЛА ОТВЕТОВ:
- Всегда отвечай на русском языке
- Будь кратким и деловым (3-5 предложений)  
- Не указывай источники при использовании RAG
- Ссылки в веб-поиске - только при явном запросе
- После каждого ответа предлагай следующий шаг (CTA)

ИНСТРУМЕНТЫ:
- rag_search: для поиска в наших кейсах и программах обучения
- web_search: обязательно использовать для веб-поиска по всем вопросам вне кейсов, программ обучения, портфолио и имеющейся информации о компании SMIRNOFF_AI, кроме очевидно простых вопросов
- lead_capture: для сохранения контактов клиентов

Помогай клиентам найти подходящие AI-решения для их бизнеса.""",
    checkpointer=checkpointer  # уже созданный SqliteSaver
)

logger.info("LangChain агент создан ✅")


In [ ]:
# Визуализация графа
from IPython.display import Image, display

# Пробуем создать PNG изображение графа
try:
    graph_image = agent_langchain.get_graph().draw_mermaid_png()
    display(Image(graph_image))
    logger.info("Граф LangChain агента визуализирован ✅")
except Exception as e:
    # Если не получилось (нет интернета или других проблем), показываем текстовую версию
    logger.warning(f"Не удалось создать PNG ({e}), показываем текстовую диаграмму")
    print("\n📊 Структура графа LangChain агента:\n")
    print(agent_langchain.get_graph().draw_mermaid())
    logger.info("Граф LangChain агента показан в текстовом формате ✅")


In [ ]:
# Хелпер функция для тестирования LangChain агента
def run_turn_langchain(chat_id: str, user_text: str):
    """Запускает один оборот диалога с LangChain агентом."""
    
    # Создаем конфиг с thread_id (как в примере)
    config = {"configurable": {"thread_id": str(chat_id)}}
    
    # Создаем сообщение пользователя
    inputs = {"messages": [{"role": "user", "content": user_text}]}
    
    print(f"\n💬 Чат {chat_id}: {user_text}")
    print("=" * 60)
    
    # Запускаем граф с режимом values для просмотра финальных сообщений
    for step in agent_langchain.stream(inputs, config=config, stream_mode="values"):
        step["messages"][-1].pretty_print()
    
    print("=" * 60)
    return config

logger.info("Хелпер функция для LangChain агента создана ✅")


In [ ]:
# 🎯 ТЕСТ: Полноценный диалог с LangChain агентом
print("=" * 80)
print("🎯 ТЕСТ: Полноценный диалог с LangChain агентом")
print("=" * 80)

client_id = "langchain_client_11"

# 📞 Шаг 1: Клиент спрашивает об услугах
print("\n📞 Шаг 1: Клиент спрашивает об услугах")
run_turn_langchain(client_id, "Привет! Что вы можете для нас сделать?")


In [ ]:
# 📊 Шаг 2: Клиент просит кейсы (RAG)
print("\n📊 Шаг 2: Клиент просит кейсы")
run_turn_langchain(client_id, "Интересно. А есть успешные кейсы для медиа агентства?")


In [ ]:
# 🔍 Шаг 3: Клиент задает технический вопрос (веб-поиск)
print("\n🔍 Шаг 3: Клиент задает технический вопрос")
run_turn_langchain(client_id, "А правда, что вышла новая версия Langchain 1.0 для production применения?")


In [ ]:
# 💼 Шаг 4: Клиент оставляет контакты (lead_capture)
print("\n💼 Шаг 4: Клиент оставляет контакты")
run_turn_langchain(client_id, "Понятно. Меня зовут Мария, maria@startup.io, хочу обсудить автоматизацию техподдержки")


## Сравнение подходов: LangGraph vs create_agent

| Аспект | LangGraph (ручной) | create_agent (LangChain 1.0) |
|--------|-------------------|------------------------------|
| **Количество кода** | ~200 строк | ~20 строк |
| **Гибкость** | Полный контроль над графом | Ограничена стандартным API |
| **Сложность** | Высокая | Низкая |
| **Время разработки** | Больше | Меньше |
| **Кастомизация** | Любые узлы и рёбра | Только через middleware |
| **Отладка** | Явная логика каждого шага | Скрытая внутри create_agent |
| **Когда использовать** | Сложная логика, кастомные узлы, нестандартные потоки | Стандартные ReAct агенты, быстрый прототип, простые задачи |

**Вывод:**
- Для прототипирования и стандартных агентов → `create_agent()`
- Для продакшена с нестандартной логикой → ручной LangGraph

**В нашем случае:** Оба подхода работают идентично, но `create_agent()` существенно проще в использовании для данной задачи.
